<a href="https://colab.research.google.com/github/Junten/context-graph/blob/master/src/Polynomial%20Equation/dynamodb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
AWS_ACCESS_KEY_ID = " "
AWS_SECRET_ACCESS_KEY = " "
AWS_REGION = 'us-east-1'

In [0]:
import uuid
import boto3
from datetime import datetime
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

class DynamoDB(object):
    def __init__(self, api_key, secret_key, table_name, aws_region='us-east-1'):
        try:
            self.__session = boto3.Session(
                aws_access_key_id = api_key,
                aws_secret_access_key = secret_key,
                region_name = aws_region,
            )
            self.__dynamodb = self.__session.resource('dynamodb')
            self.__table = self.__dynamodb.Table(table_name)
        except ClientError as e:
            print(e.response['Error']['Message'])

    def _convert_timestamp(self, response_data):
        for item in response_data:
            time_str = datetime.fromtimestamp(item['timestamp']).strftime('%Y-%m-%d')
            item['timestamp'] = time_str
        return response_data

    def _convert_timestring(self, timestring):
        for format in ['%Y-%m-%d', '%Y/%m/%d', '%Y%m%d', '%Y %m %d']:
            try:
                timestamp = datetime.strptime(timestring, format).strftime('%s')
                return int(timestamp)
            except ValueError as e:
                pass

    def insert(self, **kwargs):
        item_id = uuid.uuid4().hex
        if 'item_id' not in kwargs.keys():
            kwargs['item_id'] = item_id
        if 'timestamp' not in kwargs.keys():
            kwargs['timestamp'] = 0
        else:
            kwargs['timestamp'] = self._convert_timestring(kwargs['timestamp'])
        try:
            response = table.put_item(
                Item=kwargs
            )
            return response['ResponseMetadata']
        except ClientError as e:
            return e.response['Error']['Message']

    def search_all(self):
        try:
            response = self.__table.scan()
            data = response['Items']
            while 'LastEvaluatedKey' in response:
                response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
                data.extend(response['Items'])
            return data
        except ClientError as e:
            return e.response['Error']['Message']

    def search_by_id(self, item_id):
        try:
            response = slef.__table.query(
                KeyConditionExpression=Key('item_id').eq(item_id),
                ScanIndexForward=True
            )
            return response['Items']
        except ClientError as e:
            return e.response['Error']['Message']            

    def search_by_timestamp(self, timestamp):
        timestamp = self._convert_timestring(timestamp)
        try:
            filtering_exp = Key('timestamp').eq(timestamp)
            response = self.__table.scan(
                FilterExpression=filtering_exp
            )
            return response['Items']
        except ClientError as e:
            return e.response['Error']['Message']

    def search_by_timestamp_range(self, start=None, end=None):
        if not start:
            start = 0
        else:
            start = self._convert_timestring(start)
        if not end:
            end = int(datetime.now().strftime('%s'))
        else:
            end = self._convert_timestring(end)
        try:
            filtering_exp = Key('timestamp').between(start, end)
            response = self.__table.scan(
                FilterExpression=filtering_exp
            )
            return response['Items']
        except ClientError as e:
            return e.response['Error']['Message']

    def delete_by_id(self, item_id):
        records = self.search_by_id(item_id)
        if not records:
            return "No Records Found"
        try:
            response = self.__table.delete_item(
                Key = {
                    'item_id': records[0]['item_id'],
                    'timestamp': records[0]['timestamp']
                }
            )
            return response['ResponseMetadata']
        except ClientError as e:
            return e.response['Error']['Message']